#### Se requiere obtener informacion de "peliculas", el "Pais" donde se realizo la grabacion y cual fue la "productora" encargada de realizarlo.
#### La informacion debe ser apartir de año 2010 en adelante de la fecha de lanzamiento, ordenado de  manera descendente por el "titulo" de la pelicula

- De la pelicula se requiere mostrar el "titulo", el "presupuesto", los "ingresos obtenidos", "tiempo de duracion" y "fecha de lanzamiento"
- De "Pais" se requiere mostrar el "nombre del pais"
- De "Productora" se requiere el "nombre de la productora"

#### Leer todos los datos que son requeridos

##### Estas son las DataFrames o Tablas a leer
- movie
- country
- production_country
- movie_company
- production_company

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
#movies_df = spark.read.parquet(f"{silver_folder_path}/movies") \
#            .filter(f"file_date = '{v_file_date}'")

#Modulo: Delta Lake
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
            .filter(f"file_date = '{v_file_date}'")

In [0]:
#country_df = spark.read.parquet(f"{silver_folder_path}/countries")

# Modulo: Delta Lake
country_df = spark.read.format("delta").load(f"{silver_folder_path}/countries")

In [0]:
#movie_company_df = spark.read.parquet(f"{silver_folder_path}/movies_companies") \
#            .filter(f"file_date = '{v_file_date}'")
      
#Modulo: Delta Lake
movie_company_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_companies") \
            .filter(f"file_date = '{v_file_date}'")

In [0]:
#production_country_df = spark.read.parquet(f"{silver_folder_path}/productions_countries") \
#            .filter(f"file_date = '{v_file_date}'")
      
#Modulo: Delta Lake
production_country_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_countries") \
            .filter(f"file_date = '{v_file_date}'")

In [0]:
#production_company_df = spark.read.parquet(f"{silver_folder_path}/productions_companies") \
#            .filter(f"file_date = '{v_file_date}'")

#Modulo: Delta Lake
production_company_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_companies") \
            .filter(f"file_date = '{v_file_date}'")

#### Join "country" y "production_country"

In [0]:
country_prod_coun_df = country_df.join(production_country_df,
                                         country_df.country_id == production_country_df.country_id,
                                         'inner') \
                        .select(country_df.country_name,country_df.country_id, production_country_df.movie_id)

#### Join "production_company" y "movie_company"

In [0]:
prod_comp_mov_comp_df = production_company_df.join(movie_company_df,
                                                         production_company_df.company_id == movie_company_df.company_id,
                                                         "inner") \
                                .select(production_company_df.company_name,production_company_df.company_id, movie_company_df.movie_id)

#### Join "movies_df", "country_prod_coun_df" y "prod_comp_mov_comp_df"

- Filtrar las peliculas donde su fecha de lanzamiento sea mahor o igual a 2010

In [0]:
#Usamos expresion SQL
movie_filter_df = movies_df.filter("year_release_date >=2010")

In [0]:
result_movies_country_prod_company_df = movie_filter_df.join(country_prod_coun_df,
                                                 movie_filter_df.movie_id == country_prod_coun_df.movie_id,
                                                 "inner") \
                                                     .join(prod_comp_mov_comp_df,
                                                movie_filter_df.movie_id == prod_comp_mov_comp_df.movie_id,
                                                "inner")

- Agregar columna "created_date"

In [0]:
from pyspark.sql.functions import lit

In [0]:
result_df = result_movies_country_prod_company_df \
            .select(movie_filter_df.movie_id,"country_id","company_id","title","budget","revenue","duration_time","release_date","country_name","company_name") \
            .withColumn("created_date", lit(v_file_date))

- Ordenar por la columna "title" de manera descendente

In [0]:
results_order_by_df = result_df.orderBy(result_df.title.desc())

In [0]:
display(results_order_by_df) 

#### Escribir los datos en el DataLake en formato "Delta"

In [0]:
#overwrite_partition(results_order_by_df,"movie_gold","results_country_prod_company","created_date")

In [0]:
# results_order_by_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_country_prod_company")

# Crear una tabla en base al data frame results_order_by_df
# Creara la tabla results_country_prod_company en a DB que se llame movie_gold
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Gold

#results_order_by_df.write.mode("overwrite").format("parquet").saveAsTable("movie_gold.results_country_prod_company")

#Nota: Esto es una tabla administrada por Databricks, esto quiere decir que si la tabla se elimina, los registros tambien se van a eliminar
# Append, agrega los registros a la tabla existente, carga incremental de datos
#results_order_by_df.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_country_prod_company")


#Modulo: Delta Lake
merge_condition = "tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.company_id = src.company_id AND tgt.created_date = src.created_date" # Buscar por particion
merge_delta_lake(results_order_by_df,"movie_gold","results_country_prod_company",gold_folder_path,merge_condition,"created_date")

In [0]:
%sql
-- Consultar la DB movie_gold, tabla results_country_prod_company
SELECT created_date, COUNT(1) FROM 
movie_gold.results_country_prod_company
GROUP BY 1;

In [0]:
#display(spark.read.parquet(f"{gold_folder_path}/results_country_prod_company"))
display(spark.read.format("delta").load(f"{gold_folder_path}/results_country_prod_company")) 